In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 98% 324M/331M [00:02<00:00, 140MB/s]
100% 331M/331M [00:02<00:00, 125MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [5]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [49]:
age=[]
gender=[]
file_name=[]

for i in os.listdir(folder_path):
  age.append(i.split('_')[0])
  gender.append(i.split('_')[1])
  file_name.append(i)

In [50]:
len(age)

23708

In [51]:
df = pd.DataFrame({'age':age,'gender':gender,'img':file_name})

In [52]:
df.head()

,age,gender,img
0,30,1,30_1_0_20170109001620649.jpg.chip.jpg
1,40,0,40_0_2_20170104192449111.jpg.chip.jpg
2,12,1,12_1_0_20170109212505669.jpg.chip.jpg
3,27,1,27_1_2_20170116170654971.jpg.chip.jpg
4,40,0,40_0_1_20170117170632910.jpg.chip.jpg


In [73]:
train_df = df.sample(frac=1, random_state=0).iloc[:20000]
test_df = df.sample(frac=1, random_state=0).iloc[20000:]

In [74]:
train_df.shape

(20000, 3)

In [75]:
test_df.shape

(3708, 3)

In [76]:
train_df['age'] = train_df['age'].astype(np.float32) # Ensure age is numeric
train_df['gender'] = train_df['gender'].astype(np.float32) # Ensure gender is numeric

test_df['age'] = test_df['age'].astype(np.float32) # Ensure age is numeric
test_df['gender'] = test_df['gender'].astype(np.float32) # Ensure gender is numeric


In [77]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [78]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [65]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [79]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [80]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [81]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
model.summary()

In [85]:
def train_generator_fn():
    for batch_x, batch_y in train_generator:
        # Accessing the age and gender arrays individually from the list
        yield batch_x, {'age': batch_y[0], 'gender': batch_y[1]}

def test_generator_fn():
    for batch_x, batch_y in test_generator:
        # Accessing the age and gender arrays individually from the list
        yield batch_x, {'age': batch_y[0], 'gender': batch_y[1]}

In [86]:
model.fit(train_generator_fn(),
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=test_generator_fn(),
                    validation_steps=len(test_generator)
                    )

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 237s 357ms/step - age_loss: 16.1803 - age_mae: 16.1803 - gender_accuracy: 0.5113 - gender_loss: 125.1844 - loss: 141.3648 - val_age_loss: 15.6008 - val_age_mae: 15.6051 - val_gender_accuracy: 0.5299 - val_gender_loss: 68.5047 - val_loss: 84.1096
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 211s 339ms/step - age_loss: 15.0681 - age_mae: 15.0681 - gender_accuracy: 0.5189 - gender_loss: 68.5847 - loss: 83.6528 - val_age_loss: 14.9461 - val_age_mae: 14.9473 - val_gender_accuracy: 0.5299 - val_gender_loss: 68.4548 - val_loss: 83.4022
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 211s 339ms/step - age_loss: 14.7558 - age_mae: 14.7558 - gender_accuracy: 0.5212 - gender_loss: 68.6042 - loss: 83.3601 - val_age_loss: 14.7806 - val_age_mae: 14.7769 - val_gender_accuracy: 0.5299 - val_gender_loss: 68.4449 - val_loss: 83.2223
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 214s 343ms/step - age_loss: 14.8516 - age_mae: 14.8516 - gender_accuracy: 0.5193 - gender_loss: 68.8230 - 

KeyboardInterrupt: 

In [ ]:
# model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)